In [14]:
import pandas as pd
import os
import sys
import boto3
import psycopg2
import ast

sys.path.append(os.path.abspath(os.path.join(sys.path[0], '..', 'src', 'lib')))
import athena_connect

In [12]:
def logic_db_connection():
    """Fetches Logic DB postgres connection object

    Returns:
        A database connection object for Postgres
    """
    try:
        boto_session = boto3.Session(profile_name='loidsig')
    except:
        boto_session = boto3.Session()
    sm_client = boto_session.client(
        service_name='secretsmanager',
        region_name='us-east-1',
        endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
    )
    get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_DB')
    cred_dict = ast.literal_eval(get_secret_value_response['SecretString'])
    db_user, db_pass = cred_dict['username'], cred_dict['password']
    db_host, db_port, db_name = cred_dict['host'], cred_dict['port'], cred_dict['dbname']

    try:
        conn = psycopg2.connect(
            host=db_host,
            port=db_port,
            user=db_user,
            password=db_pass,
            database=db_name,
        )
    except Exception as e:
        print("Unable to connect to postgres! Error: {}".format(e))
        raise
    return conn
    
def pandas_read_postgres(sql_query):
    """Given a Redshift SQL query, this method returns a pandas dataframe

    Args:
        sql_query: a Presto sql query (use the Hue Presto notebook interface to validate sql)

    Returns:
        A pandas dataframe
    """
    logic_db_conn = logic_db_connection()
    df = pd.read_sql_query(sql_query, logic_db_conn)
    logic_db_conn.close()
    return df

# Cobinhood Buy

In [37]:
athena = athena_connect.Athena()
sql = """SELECT *, (CAST(open_timestamp AS BIGINT) / 1000 / 60) as trade_minute
        FROM cobinhood.historic_candlesticks 
        WHERE coin_partition = 'btcusdt'
        AND DATE(from_unixtime(cast(open_timestamp AS BIGINT) / 1000)) > DATE('2018-09-01')"""
reported_df = athena.pandas_read_athena(sql)
reported_df

,row_id,close,high,low,open,time_window,open_timestamp,coin,volume,open_unix,open_datetime,file_name,coin_partition,trade_minute
0,0,7359.4,7359.4,7359.4,7359.4,1m,1536105600000,BTCUSDT,0.000000,1.536106e+09,2018-09-05 00:00:00,btcusdt/2018-09-05.csv,btcusdt,25601760
1,1,7354.3,7354.3,7354.3,7354.3,1m,1536105660000,BTCUSDT,0.120858,1.536106e+09,2018-09-05 00:01:00,btcusdt/2018-09-05.csv,btcusdt,25601761
2,2,7354.3,7354.3,7354.3,7354.3,1m,1536105720000,BTCUSDT,0.000000,1.536106e+09,2018-09-05 00:02:00,btcusdt/2018-09-05.csv,btcusdt,25601762
3,3,7354.3,7354.3,7354.3,7354.3,1m,1536105780000,BTCUSDT,0.000000,1.536106e+09,2018-09-05 00:03:00,btcusdt/2018-09-05.csv,btcusdt,25601763
4,4,7354.3,7354.3,7354.3,7354.3,1m,1536105840000,BTCUSDT,0.000000,1.536106e+09,2018-09-05 00:04:00,btcusdt/2018-09-05.csv,btcusdt,25601764
5,5,7350.7,7350.7,7350.7,7350.7,1m,1536105900000,BTCUSDT,0.309345,1.536106e+09,2018-09-05 00:05:00,btcusdt/2018-09-05.csv,btcusdt,25601765
6,6,7363.3,7363.3,7363.2,7363.2,1m,1536105960000,BTCUSDT,0.112300,1.536106e+09,2018-09-05 00:06:00,btcusdt/2018-09-05.csv,btcusdt,25601766
7,7,7363.2,7363.2,7363.2,7363.2,1m,1536106020000,BTCUSDT,0.117328,1.536106e+09,2018-09-05 00:07:00,btcusdt/2018-09-05.csv,btcusdt,25601767
8,8,7363.2,7363.2,7363.2,7363.2,1m,1536106080000,BTCUSDT,0.000000,1.536106e+09,2018-09-05 00:08:00,btcusdt/2018-09-05.csv,btcusdt,25601768
9,9,7350.6,7350.6,7350.6,7350.6,1m,1536106140000,BTCUSDT,0.148432,1.536106e+09,2018-09-05 00:09:00,btcusdt/2018-09-05.csv,btcusdt,25601769


In [38]:
sql = """select * 
        from the_logic.scoring_results
        where is_trade = true
        and model_version = 2.1"""
actual_df = pandas_read_postgres(sql)
actual_df['trade_minute'] = actual_df['trade_minute'] + 1
actual_df

,trade_datetime,trade_minute,target_coin,trade_duration,predicted_return,predicted_growth_rate,highest_return,is_trade,trade_threshold,feature_window_space,...,sell_commission,buy_commission_coin,sell_commission_coin,buy_order_id,buy_client_order_id,sell_order_id,sell_client_order_id,model_version,buy_fill_latency_seconds,sell_fill_latency_seconds
0,2018-09-07 13:39:08,25605459,btcusdt,7,0.078823,0.011260,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,73e38567-cbda-4f26-bcf8-463cec3d7bab,0,8dfc578a-ba3d-4b3f-b20d-6a90f3f60c09,2.1,5,5
1,2018-09-07 13:47:29,25605467,btcusdt,7,0.082198,0.011743,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,cd2dc995-1390-4e99-96d7-bf98885272a8,0,1de5ed3b-8ac8-41cb-b9e5-6945f9a1fa64,2.1,5,5
2,2018-09-07 13:53:42,25605473,btcusdt,7,0.043764,0.006252,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,3246c560-2d30-448a-adb8-72a9150e407b,0,97a7b066-f54d-4fa0-8017-ffc5d8d92794,2.1,5,5
3,2018-09-07 15:05:20,25605545,btcusdt,7,0.076993,0.010999,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,370d9414-5804-4ceb-9ca5-2f9de970b28e,0,e1cccd55-4399-45bf-9398-bd6379f553e1,2.1,5,5
4,2018-09-07 15:47:42,25605587,btcusdt,7,0.066337,0.009477,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,1a09cca2-41a6-41a4-878a-4f7b65142b31,0,62dd7b5e-e5ba-43c4-8fcc-59eeb057e86e,2.1,5,5
5,2018-09-07 15:59:35,25605599,btcusdt,7,0.159436,0.022777,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,4e4ced92-10f2-408f-b29f-4658f8ca1b10,0,5bb51e8e-4528-472c-a6ea-1aa3485f19cc,2.1,5,5
6,2018-09-07 16:06:18,25605606,btcusdt,7,0.191399,0.027343,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,947980a4-5d3b-48d3-b597-ec0efc88d377,0,b1135e54-ac89-4bb9-97a3-97557d6f2270,2.1,5,5
7,2018-09-07 16:13:17,25605613,btcusdt,7,0.151670,0.021667,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,ba6f4c20-2d1a-4e06-b6a2-5e16b4b0d5d0,0,45d185ec-7185-44b1-883c-cbebf5b00dc9,2.1,5,5
8,2018-09-07 16:22:04,25605622,btcusdt,7,0.049557,0.007080,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,ebc4471d-f8fe-453c-8db6-60d9d93cbbfa,0,a377c8d0-faa4-4cf5-a11f-3f5d7300ab62,2.1,5,5
9,2018-09-07 16:28:43,25605628,btcusdt,7,0.052771,0.007539,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,2fdc73a2-42f2-4961-932e-e7a8e1ef230f,0,d909e9f0-08ae-444b-9798-e95eae5b11f4,2.1,5,5


In [39]:
compare_df = pd.merge(actual_df, reported_df, on='trade_minute', how='left')
compare_df

,trade_datetime,trade_minute,target_coin,trade_duration,predicted_return,predicted_growth_rate,highest_return,is_trade,trade_threshold,feature_window_space,...,low,open,time_window,open_timestamp,coin,volume,open_unix,open_datetime,file_name,coin_partition
0,2018-09-07 13:39:08,25605459,btcusdt,7,0.078823,0.011260,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6429.7,6433.5,1m,1536327540000,BTCUSDT,0.490343,1.536328e+09,2018-09-07 13:39:00,btcusdt/2018-09-07.csv,btcusdt
1,2018-09-07 13:47:29,25605467,btcusdt,7,0.082198,0.011743,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6438.3,6438.3,1m,1536328020000,BTCUSDT,0.054591,1.536328e+09,2018-09-07 13:47:00,btcusdt/2018-09-07.csv,btcusdt
2,2018-09-07 13:53:42,25605473,btcusdt,7,0.043764,0.006252,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6429.4,6429.4,1m,1536328380000,BTCUSDT,0.168154,1.536328e+09,2018-09-07 13:53:00,btcusdt/2018-09-07.csv,btcusdt
3,2018-09-07 15:05:20,25605545,btcusdt,7,0.076993,0.010999,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6433.8,6433.8,1m,1536332700000,BTCUSDT,0.062684,1.536333e+09,2018-09-07 15:05:00,btcusdt/2018-09-07.csv,btcusdt
4,2018-09-07 15:47:42,25605587,btcusdt,7,0.066337,0.009477,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6417.1,6418.1,1m,1536335220000,BTCUSDT,0.343129,1.536335e+09,2018-09-07 15:47:00,btcusdt/2018-09-07.csv,btcusdt
5,2018-09-07 15:59:35,25605599,btcusdt,7,0.159436,0.022777,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6366.9,6367.2,1m,1536335940000,BTCUSDT,0.462493,1.536336e+09,2018-09-07 15:59:00,btcusdt/2018-09-07.csv,btcusdt
6,2018-09-07 16:06:18,25605606,btcusdt,7,0.191399,0.027343,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6397.8,6401.8,1m,1536336360000,BTCUSDT,0.627354,1.536336e+09,2018-09-07 16:06:00,btcusdt/2018-09-07.csv,btcusdt
7,2018-09-07 16:13:17,25605613,btcusdt,7,0.151670,0.021667,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6444.4,6444.4,1m,1536336780000,BTCUSDT,0.053805,1.536337e+09,2018-09-07 16:13:00,btcusdt/2018-09-07.csv,btcusdt
8,2018-09-07 16:22:04,25605622,btcusdt,7,0.049557,0.007080,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6450.5,6469.0,1m,1536337320000,BTCUSDT,0.266282,1.536337e+09,2018-09-07 16:22:00,btcusdt/2018-09-07.csv,btcusdt
9,2018-09-07 16:28:43,25605628,btcusdt,7,0.052771,0.007539,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6447.9,6447.9,1m,1536337680000,BTCUSDT,0.107651,1.536338e+09,2018-09-07 16:28:00,btcusdt/2018-09-07.csv,btcusdt


In [40]:
compare_df[['trade_datetime','open_datetime','predicted_return','buy_price','sell_price','close','open','high','low']]

,trade_datetime,open_datetime,predicted_return,buy_price,sell_price,close,open,high,low
0,2018-09-07 13:39:08,2018-09-07 13:39:00,0.078823,6433.500000,6425.100000,6429.7,6433.5,6433.5,6429.7
1,2018-09-07 13:47:29,2018-09-07 13:47:00,0.082198,6438.398784,6429.400000,6438.7,6438.3,6438.7,6438.3
2,2018-09-07 13:53:42,2018-09-07 13:53:00,0.043764,6440.500000,6435.200000,6440.5,6429.4,6440.5,6429.4
3,2018-09-07 15:05:20,2018-09-07 15:05:00,0.076993,6433.800000,6417.000000,6435.1,6433.8,6435.1,6433.8
4,2018-09-07 15:47:42,2018-09-07 15:47:00,0.066337,6431.800000,6418.478867,6431.8,6418.1,6431.8,6417.1
5,2018-09-07 15:59:35,2018-09-07 15:59:00,0.159436,6385.000000,6365.100000,6368.2,6367.2,6385.0,6366.9
6,2018-09-07 16:06:18,2018-09-07 16:06:00,0.191399,6399.800000,6435.200000,6420.9,6401.8,6422.9,6397.8
7,2018-09-07 16:13:17,2018-09-07 16:13:00,0.151670,6444.400000,6457.700000,6444.4,6444.4,6444.4,6444.4
8,2018-09-07 16:22:04,2018-09-07 16:22:00,0.049557,6469.000000,6447.900000,6450.5,6469.0,6469.0,6450.5
9,2018-09-07 16:28:43,2018-09-07 16:28:00,0.052771,6456.400000,6452.400000,6456.4,6447.9,6456.4,6447.9


In [49]:
compare_df[['trade_datetime','open_datetime','predicted_return','buy_price','close','open','high','low']].to_clipboard()

# Cobinhood Sell

In [42]:
athena = athena_connect.Athena()
sql = """SELECT *, (CAST(open_timestamp AS BIGINT) / 1000 / 60) as trade_minute
        FROM cobinhood.historic_candlesticks 
        WHERE coin_partition = 'btcusdt'
        AND DATE(from_unixtime(cast(open_timestamp AS BIGINT) / 1000)) > DATE('2018-09-01')"""
reported_df = athena.pandas_read_athena(sql)
reported_df

,row_id,close,high,low,open,time_window,open_timestamp,coin,volume,open_unix,open_datetime,file_name,coin_partition,trade_minute
0,0,7259.4,7259.4,7259.4,7259.4,1m,1536019200000,BTCUSDT,0.115057,1.536019e+09,2018-09-04 00:00:00,btcusdt/2018-09-04.csv,btcusdt,25600320
1,1,7256.7,7256.7,7256.7,7256.7,1m,1536019260000,BTCUSDT,0.167907,1.536019e+09,2018-09-04 00:01:00,btcusdt/2018-09-04.csv,btcusdt,25600321
2,2,7261.1,7261.1,7257.7,7257.7,1m,1536019320000,BTCUSDT,0.363682,1.536019e+09,2018-09-04 00:02:00,btcusdt/2018-09-04.csv,btcusdt,25600322
3,3,7261.1,7261.1,7261.1,7261.1,1m,1536019380000,BTCUSDT,0.000000,1.536019e+09,2018-09-04 00:03:00,btcusdt/2018-09-04.csv,btcusdt,25600323
4,4,7261.1,7261.1,7261.1,7261.1,1m,1536019440000,BTCUSDT,0.000000,1.536019e+09,2018-09-04 00:04:00,btcusdt/2018-09-04.csv,btcusdt,25600324
5,5,7261.1,7261.1,7261.1,7261.1,1m,1536019500000,BTCUSDT,0.000000,1.536020e+09,2018-09-04 00:05:00,btcusdt/2018-09-04.csv,btcusdt,25600325
6,6,7261.1,7261.1,7261.1,7261.1,1m,1536019560000,BTCUSDT,0.000000,1.536020e+09,2018-09-04 00:06:00,btcusdt/2018-09-04.csv,btcusdt,25600326
7,7,7265.5,7265.5,7265.5,7265.5,1m,1536019620000,BTCUSDT,0.220662,1.536020e+09,2018-09-04 00:07:00,btcusdt/2018-09-04.csv,btcusdt,25600327
8,8,7266.1,7266.8,7266.1,7266.8,1m,1536019680000,BTCUSDT,0.212082,1.536020e+09,2018-09-04 00:08:00,btcusdt/2018-09-04.csv,btcusdt,25600328
9,9,7266.1,7266.1,7266.1,7266.1,1m,1536019740000,BTCUSDT,0.000000,1.536020e+09,2018-09-04 00:09:00,btcusdt/2018-09-04.csv,btcusdt,25600329


In [43]:
sql = """select * 
        from the_logic.scoring_results
        where is_trade = true
        and model_version = 2.1"""
actual_df = pandas_read_postgres(sql)
actual_df['trade_minute'] = actual_df['trade_minute'] + 7
actual_df

,trade_datetime,trade_minute,target_coin,trade_duration,predicted_return,predicted_growth_rate,highest_return,is_trade,trade_threshold,feature_window_space,...,sell_commission,buy_commission_coin,sell_commission_coin,buy_order_id,buy_client_order_id,sell_order_id,sell_client_order_id,model_version,buy_fill_latency_seconds,sell_fill_latency_seconds
0,2018-09-07 13:39:08,25605465,btcusdt,7,0.078823,0.011260,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,73e38567-cbda-4f26-bcf8-463cec3d7bab,0,8dfc578a-ba3d-4b3f-b20d-6a90f3f60c09,2.1,5,5
1,2018-09-07 14:13:14,25605499,btcusdt,7,0.044121,0.006303,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,c7f04199-7a40-4abd-8b08-49c415a734fa,0,d4611902-a748-45bb-b6c5-14d592be2dc5,2.1,5,5
2,2018-09-07 14:19:42,25605505,btcusdt,7,0.050760,0.007251,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,78670e5e-4eb2-430d-b32e-827e70eef624,0,89998491-4786-4bbf-8062-37a62bb97277,2.1,5,5
3,2018-09-07 14:26:17,25605512,btcusdt,7,0.080408,0.011487,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,1152025f-2d6c-4020-8bca-727c8ee64d66,0,ae5b8672-b8d8-4e7f-9ed9-f63121c3f8f5,2.1,5,5
4,2018-09-07 14:33:18,25605519,btcusdt,7,0.067426,0.009632,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,2e3fd665-d8af-4e1e-af7d-bcd6f77e0384,0,c969852e-bf90-4f68-950c-21a3ba005892,2.1,5,5
5,2018-09-07 14:45:01,25605531,btcusdt,7,0.075685,0.010812,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,76359c78-1c83-40e7-9d85-5b2d97426ead,0,fb267263-2511-4370-adc5-2346b465fa45,2.1,5,5
6,2018-09-07 15:05:20,25605551,btcusdt,7,0.076993,0.010999,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,370d9414-5804-4ceb-9ca5-2f9de970b28e,0,e1cccd55-4399-45bf-9398-bd6379f553e1,2.1,5,5
7,2018-09-07 15:19:22,25605565,btcusdt,7,0.111311,0.015902,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,fc826a6f-877e-4e11-8c3c-a27589c539ac,0,35eba448-5b3b-4d04-9f5d-881d55fc0fe3,2.1,5,5
8,2018-09-07 15:26:19,25605572,btcusdt,7,0.133159,0.019023,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,3742e538-f234-4e0f-b55f-6fd0ef737c6d,0,678cb0e0-0d04-4fc7-b9f1-80799ffc4ea3,2.1,5,5
9,2018-09-07 15:41:32,25605587,btcusdt,7,0.057615,0.008231,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,0.0,,,0,44ce15f8-011d-49b0-9fd4-37f6945a8717,0,7b7f5360-cb74-4d1f-8d78-7039f89ca7e1,2.1,5,5


In [44]:
compare_df = pd.merge(actual_df, reported_df, on='trade_minute', how='left')
compare_df

,trade_datetime,trade_minute,target_coin,trade_duration,predicted_return,predicted_growth_rate,highest_return,is_trade,trade_threshold,feature_window_space,...,low,open,time_window,open_timestamp,coin,volume,open_unix,open_datetime,file_name,coin_partition
0,2018-09-07 13:39:08,25605465,btcusdt,7,0.078823,0.011260,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6422.6,6425.1,1m,1536327900000,BTCUSDT,0.653147,1.536328e+09,2018-09-07 13:45:00,btcusdt/2018-09-07.csv,btcusdt
1,2018-09-07 14:13:14,25605499,btcusdt,7,0.044121,0.006303,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6436.1,6436.1,1m,1536329940000,BTCUSDT,0.125176,1.536330e+09,2018-09-07 14:19:00,btcusdt/2018-09-07.csv,btcusdt
2,2018-09-07 14:19:42,25605505,btcusdt,7,0.050760,0.007251,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6454.8,6454.8,1m,1536330300000,BTCUSDT,0.508171,1.536330e+09,2018-09-07 14:25:00,btcusdt/2018-09-07.csv,btcusdt
3,2018-09-07 14:26:17,25605512,btcusdt,7,0.080408,0.011487,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6454.6,6454.6,1m,1536330720000,BTCUSDT,0.054420,1.536331e+09,2018-09-07 14:32:00,btcusdt/2018-09-07.csv,btcusdt
4,2018-09-07 14:33:18,25605519,btcusdt,7,0.067426,0.009632,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6445.6,6445.6,1m,1536331140000,BTCUSDT,0.054095,1.536331e+09,2018-09-07 14:39:00,btcusdt/2018-09-07.csv,btcusdt
5,2018-09-07 14:45:01,25605531,btcusdt,7,0.075685,0.010812,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6432.1,6432.1,1m,1536331860000,BTCUSDT,0.053946,1.536332e+09,2018-09-07 14:51:00,btcusdt/2018-09-07.csv,btcusdt
6,2018-09-07 15:05:20,25605551,btcusdt,7,0.076993,0.010999,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6417.0,6417.0,1m,1536333060000,BTCUSDT,0.280129,1.536333e+09,2018-09-07 15:11:00,btcusdt/2018-09-07.csv,btcusdt
7,2018-09-07 15:19:22,25605565,btcusdt,7,0.111311,0.015902,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6411.2,6423.8,1m,1536333900000,BTCUSDT,0.503702,1.536334e+09,2018-09-07 15:25:00,btcusdt/2018-09-07.csv,btcusdt
8,2018-09-07 15:26:19,25605572,btcusdt,7,0.133159,0.019023,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6426.2,6426.2,1m,1536334320000,BTCUSDT,0.053682,1.536334e+09,2018-09-07 15:32:00,btcusdt/2018-09-07.csv,btcusdt
9,2018-09-07 15:41:32,25605587,btcusdt,7,0.057615,0.008231,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,6417.1,6418.1,1m,1536335220000,BTCUSDT,0.343129,1.536335e+09,2018-09-07 15:47:00,btcusdt/2018-09-07.csv,btcusdt


In [45]:
compare_df[['trade_datetime','open_datetime','predicted_return','sell_price','close','open','high','low']]

,trade_datetime,open_datetime,predicted_return,sell_price,close,open,high,low
0,2018-09-07 13:39:08,2018-09-07 13:45:00,0.078823,6425.100000,6422.6,6425.1,6428.4,6422.6
1,2018-09-07 14:13:14,2018-09-07 14:19:00,0.044121,6436.100000,6467.7,6436.1,6467.7,6436.1
2,2018-09-07 14:19:42,2018-09-07 14:25:00,0.050760,6454.800000,6462.6,6454.8,6466.4,6454.8
3,2018-09-07 14:26:17,2018-09-07 14:32:00,0.080408,6454.600000,6454.6,6454.6,6454.6,6454.6
4,2018-09-07 14:33:18,2018-09-07 14:39:00,0.067426,6445.600000,6445.6,6445.6,6445.6,6445.6
5,2018-09-07 14:45:01,2018-09-07 14:51:00,0.075685,6432.100000,6432.1,6432.1,6432.1,6432.1
6,2018-09-07 15:05:20,2018-09-07 15:11:00,0.076993,6417.000000,6419.2,6417.0,6419.2,6417.0
7,2018-09-07 15:19:22,2018-09-07 15:25:00,0.111311,6423.800000,6411.2,6423.8,6424.3,6411.2
8,2018-09-07 15:26:19,2018-09-07 15:32:00,0.133159,6426.200000,6426.2,6426.2,6426.2,6426.2
9,2018-09-07 15:41:32,2018-09-07 15:47:00,0.057615,6418.011423,6431.8,6418.1,6431.8,6417.1


In [50]:
compare_df[['trade_datetime','open_datetime','predicted_return','sell_price','close','open','high','low']].to_clipboard()

# Binance Comparison

In [46]:
athena = athena_connect.Athena()
sql = """SELECT *, (CAST(open_timestamp AS BIGINT) / 1000 / 60) as trade_minute
        FROM binance.historic_candlesticks 
        WHERE coin_partition = 'btcusdt'
        AND DATE(from_unixtime(cast(open_timestamp AS BIGINT) / 1000)) > DATE('2018-09-01')"""
reported_df = athena.pandas_read_athena(sql)
reported_df

,row_id,open_timestamp,open,high,low,close,volume,close_timestamp,quote_asset_volume,trade_count,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,open_unix,open_datetime,close_datetime,coin,coin_partition,trade_minute
0,0,1535846400000,7201.57,7204.82,7194.31,7204.44,44.525605,1535846459999,320675.810573,177,26.000364,187276.353009,0,1.535846e+09,2018-09-02 00:00:00.000,2018-09-02 00:00:59.999,BTCUSDT,btcusdt,25597440
1,1,1535846460000,7204.47,7204.82,7201.62,7203.99,15.758991,1535846519999,113533.789426,105,9.128374,65764.011272,0,1.535846e+09,2018-09-02 00:01:00.000,2018-09-02 00:01:59.999,BTCUSDT,btcusdt,25597441
2,2,1535846520000,7204.00,7210.00,7201.65,7204.74,7.558039,1535846579999,54457.642132,133,5.938745,42795.122404,0,1.535847e+09,2018-09-02 00:02:00.000,2018-09-02 00:02:59.999,BTCUSDT,btcusdt,25597442
3,3,1535846580000,7203.87,7208.94,7201.07,7203.98,4.458300,1535846639999,32115.357857,85,2.284938,16460.662798,0,1.535847e+09,2018-09-02 00:03:00.000,2018-09-02 00:03:59.999,BTCUSDT,btcusdt,25597443
4,4,1535846640000,7203.16,7206.26,7203.07,7203.13,64.235344,1535846699999,462697.186019,82,9.508475,68492.120332,0,1.535847e+09,2018-09-02 00:04:00.000,2018-09-02 00:04:59.999,BTCUSDT,btcusdt,25597444
5,5,1535846700000,7203.14,7206.13,7203.13,7203.17,22.462436,1535846759999,161815.614859,72,6.471116,46619.839171,0,1.535847e+09,2018-09-02 00:05:00.000,2018-09-02 00:05:59.999,BTCUSDT,btcusdt,25597445
6,6,1535846760000,7203.21,7206.86,7203.15,7203.20,46.278081,1535846819999,333372.753657,107,14.737012,106169.208130,0,1.535847e+09,2018-09-02 00:06:00.000,2018-09-02 00:06:59.999,BTCUSDT,btcusdt,25597446
7,7,1535846820000,7203.21,7204.93,7203.14,7203.14,31.559156,1535846879999,227335.996849,120,21.563520,155335.278737,0,1.535847e+09,2018-09-02 00:07:00.000,2018-09-02 00:07:59.999,BTCUSDT,btcusdt,25597447
8,8,1535846880000,7203.15,7203.15,7194.31,7195.90,51.601939,1535846939999,371378.658510,167,33.503675,241130.197436,0,1.535847e+09,2018-09-02 00:08:00.000,2018-09-02 00:08:59.999,BTCUSDT,btcusdt,25597448
9,9,1535846940000,7197.12,7201.60,7197.12,7200.40,25.828553,1535846999999,185961.460068,84,14.068495,101293.816049,0,1.535847e+09,2018-09-02 00:09:00.000,2018-09-02 00:09:59.999,BTCUSDT,btcusdt,25597449


In [47]:
compare_df = pd.merge(actual_df, reported_df, on='trade_minute', how='left')
compare_df

,trade_datetime,trade_minute,target_coin,trade_duration,predicted_return,predicted_growth_rate,highest_return,is_trade,trade_threshold,feature_window_space,...,quote_asset_volume,trade_count,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,open_unix,open_datetime,close_datetime,coin,coin_partition
0,2018-09-07 13:39:08,25605465,btcusdt,7,0.078823,0.011260,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,178910.208423,95,14.189357,91242.377322,0,1.536328e+09,2018-09-07 13:45:00.000,2018-09-07 13:45:59.999,BTCUSDT,btcusdt
1,2018-09-07 14:13:14,25605499,btcusdt,7,0.044121,0.006303,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,816184.362672,600,91.203956,588687.519119,0,1.536330e+09,2018-09-07 14:19:00.000,2018-09-07 14:19:59.999,BTCUSDT,btcusdt
2,2018-09-07 14:19:42,25605505,btcusdt,7,0.050760,0.007251,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,212473.716974,172,21.401974,138427.227983,0,1.536330e+09,2018-09-07 14:25:00.000,2018-09-07 14:25:59.999,BTCUSDT,btcusdt
3,2018-09-07 14:26:17,25605512,btcusdt,7,0.080408,0.011487,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,151282.096761,143,12.946419,83642.415686,0,1.536331e+09,2018-09-07 14:32:00.000,2018-09-07 14:32:59.999,BTCUSDT,btcusdt
4,2018-09-07 14:33:18,25605519,btcusdt,7,0.067426,0.009632,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,242435.237189,116,19.350294,124820.687632,0,1.536331e+09,2018-09-07 14:39:00.000,2018-09-07 14:39:59.999,BTCUSDT,btcusdt
5,2018-09-07 14:45:01,25605531,btcusdt,7,0.075685,0.010812,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,173604.021453,147,13.218383,85023.155558,0,1.536332e+09,2018-09-07 14:51:00.000,2018-09-07 14:51:59.999,BTCUSDT,btcusdt
6,2018-09-07 15:05:20,25605551,btcusdt,7,0.076993,0.010999,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,112962.468603,115,8.099428,52039.682890,0,1.536333e+09,2018-09-07 15:11:00.000,2018-09-07 15:11:59.999,BTCUSDT,btcusdt
7,2018-09-07 15:19:22,25605565,btcusdt,7,0.111311,0.015902,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,313745.690539,92,18.283758,117508.244827,0,1.536334e+09,2018-09-07 15:25:00.000,2018-09-07 15:25:59.999,BTCUSDT,btcusdt
8,2018-09-07 15:26:19,25605572,btcusdt,7,0.133159,0.019023,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,177529.350831,157,7.860342,50498.469165,0,1.536334e+09,2018-09-07 15:32:00.000,2018-09-07 15:32:59.999,BTCUSDT,btcusdt
9,2018-09-07 15:41:32,25605587,btcusdt,7,0.057615,0.008231,True,True,0.04,"[1, 3, 5, 10, 20, 30, 40, 50, 60, 120, 240, 48...",...,117484.544304,89,6.965816,44789.386226,0,1.536335e+09,2018-09-07 15:47:00.000,2018-09-07 15:47:59.999,BTCUSDT,btcusdt


In [48]:
compare_df[['trade_datetime','open_datetime','predicted_return','buy_price','close','open','high','low']]

,trade_datetime,open_datetime,predicted_return,buy_price,close,open,high,low
0,2018-09-07 13:39:08,2018-09-07 13:45:00.000,0.078823,6433.500000,6429.99,6428.68,6431.65,6428.66
1,2018-09-07 14:13:14,2018-09-07 14:19:00.000,0.044121,6436.700000,6464.07,6444.00,6465.00,6444.00
2,2018-09-07 14:19:42,2018-09-07 14:25:00.000,0.050760,6462.240769,6468.45,6459.41,6475.00,6459.41
3,2018-09-07 14:26:17,2018-09-07 14:32:00.000,0.080408,6468.900000,6463.28,6459.04,6466.28,6454.91
4,2018-09-07 14:33:18,2018-09-07 14:39:00.000,0.067426,6465.400000,6449.69,6452.44,6454.58,6446.00
5,2018-09-07 14:45:01,2018-09-07 14:51:00.000,0.075685,6447.700000,6426.75,6437.00,6437.50,6426.00
6,2018-09-07 15:05:20,2018-09-07 15:11:00.000,0.076993,6433.800000,6422.02,6424.38,6427.70,6421.92
7,2018-09-07 15:19:22,2018-09-07 15:25:00.000,0.111311,6426.500000,6427.90,6427.89,6427.90,6425.05
8,2018-09-07 15:26:19,2018-09-07 15:32:00.000,0.133159,6431.600000,6428.54,6430.04,6433.03,6419.11
9,2018-09-07 15:41:32,2018-09-07 15:47:00.000,0.057615,6426.010085,6431.93,6429.18,6431.93,6428.33
